<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Student 1:</b> Camilo Gabriel ROMERO SANCHEZ 
<b> Student 2:</b> Daniel HOMS ALFOSIN  
<b> Group name:</b> deeplearn6
  
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
from datetime import timedelta

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288146603
Epoch:  02   =====> Loss= 0.732879841
Epoch:  03   =====> Loss= 0.600437544
Epoch:  04   =====> Loss= 0.536478129
Epoch:  05   =====> Loss= 0.497883815
Epoch:  06   =====> Loss= 0.471082701
Epoch:  07   =====> Loss= 0.451253696
Epoch:  08   =====> Loss= 0.436073195
Epoch:  09   =====> Loss= 0.423407513
Epoch:  10   =====> Loss= 0.413047058
Epoch:  11   =====> Loss= 0.404533125
Epoch:  12   =====> Loss= 0.396643906
Epoch:  13   =====> Loss= 0.390263800
Epoch:  14   =====> Loss= 0.384461008
Epoch:  15   =====> Loss= 0.379242165
Epoch:  16   =====> Loss= 0.374706218
Epoch:  17   =====> Loss= 0.370245252
Epoch:  18   =====> Loss= 0.366504482
Epoch:  19   =====> Loss= 0.362957360
Epoch:  20   =====> Loss= 0.359751847
Epoch:  21   =====> Loss= 0.356848331
Epoch:  22   =====> Loss= 0.353783526
Epoch:  23   =====> Loss= 0.351158460
Epoch:  24   =====> Loss= 0.348740204
Epoch:  25   =====> Loss= 0.346141227
Epoch:  26   =====> Loss= 0.344057238
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0., shape=[shape])
    return tf.Variable(initial)

In [4]:
'''Definition parameters of CNN'''

# Conv layer 1
filter_size_1 = 5
num_filters_1 = 6
padding_1 = 'SAME'

# Conv layer 2
filter_size_2 = 5
num_filters_2 = 16
padding_2 = 'VALID'

# Conv layer 3
fc_size_1 = 120

# Conv layer 4
fc_size_2 = 84

In [5]:
# MNIST images size in pixel.
img_size = 28

# One-dimensional arrays of images.
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

# Colour channels (grey-scale).
num_channels = 1

# Number of classes equal to 10 digits.
num_classes = 10

In [6]:
def LeNet5_Model(input,                # Input layer.
                 num_input_channels,   # Channels in previous layer.
                 filter_size,          # Width and height filter.
                 num_filters,          # Number of filters.
                 padding,              # Padding approach.
                 activ_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Initialize weight given shape.
    weights = weight_variable(shape=shape)

    # Initialize bias given shape, one for each filter.
    biases = bias_variable(shape=num_filters)

    # Convolution operation.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding=padding)

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling according to LeNet-5 architecture.
    if activ_pooling:
        # This is 2x2 max-pooling, which applies
        # in all cases of CNN architecture.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='VALID')

    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)

    return layer, weights

In [7]:
def flatten_layer(layer):
    # Shape of the input layer.
    layer_shape = layer.get_shape()

    # Number of features
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

In [8]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs):   # Num. outputs.

    # Create new weights and biases.
    weights = weight_variable(shape=[num_inputs, num_outputs])
    biases = bias_variable(shape=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases
    
    # Passing ReLu for non linear operation.
    layer = tf.nn.relu(layer)

    return layer

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

** Layer S1 **<br>
In the first layer we have 5 size filer, so there are (filer_size x filer_size + bias) x #_filters <br>
(5x5+1)*6 = 156 parameters to lean. <br>
Assuming all connections, there will be: <br>
Total connetions: 28x28x156 = 122304<br>

** Layer S2 **<br>
(6x2) = 12 parameters to lean. <br>

** Layer S3 **<br>
5x5x(6x3 + 9x4 +1x6) + 16 = 1516 parameters to lean. <br>

** Layer S4 **<br>
(16x2) = 32 parameters to lean. <br>

** Layer C5 **<br>
120x(16x25+1) =	48120 parameters to lean. <br>

** Layer F6 **<br>
84x(120+1) = 10164 parameters to lean. <br>

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [9]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128

In [10]:
'''Input variables'''
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

'''CNN Tensorflow architecture'''
# Convolutional layer 2
layer_conv1, weights_conv1 = LeNet5_Model(input=x_image,
                                           num_input_channels=num_channels,
                                           filter_size=filter_size_1,
                                           num_filters=num_filters_1,
                                           padding=padding_1,
                                           activ_pooling=True)
# Convolutional layer 2
layer_conv2, weights_conv2 = LeNet5_Model(input=layer_conv1,
                                           num_input_channels=num_filters_1,
                                           filter_size=filter_size_2,
                                           num_filters=num_filters_2,
                                           padding=padding_2,
                                           activ_pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Fully Connectes layer 1
fc_layer_1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size_1)


# Fully Connectes layer 2
fc_layer_2 = new_fc_layer(input=fc_layer_1,
                         num_inputs=fc_size_1,
                         num_outputs=fc_size_2)


# Fully Connectes layer 3
fc_layer_3 = new_fc_layer(input=fc_layer_2,
                         num_inputs=fc_size_2,
                         num_outputs=num_classes)

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [11]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

In [12]:
def evaluate(logits, labels):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
    
    # Measuring prediction between model and real data
    difference_model_pred = tf.equal(logits, labels)
    
    # Return accurancy
    return tf.reduce_mean(tf.cast(difference_model_pred, tf.float32))

In [13]:
accuracy = evaluate(y_pred_cls, y_true_cls)

#### Implement an optimizer

In [14]:
# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3,
                                                        labels=y_true)
# Loss function.
cost = tf.reduce_mean(cross_entropy)

# Accuracy.
accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 


In [15]:
# Initializing the variables.
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor.
tf.summary.scalar('Cost_GD', cost)

# # Create a summary to monitor accuracy tensor.
# tf.summary.scalar('Accuracy_GD', accuracy)

# Merge all summaries into a single op.
merged_summary_op = tf.summary.merge_all()

In [16]:
def train(init, sess, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    
    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    for epoch in range(n_epochs):
        
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
               
            # Batch data for train_x and train_y
            x_batch, y_true_batch = mnist.train.next_batch(batch_size, 
                                                           shuffle=(i==0))

            # Feed train batch data as dic.
            feed_train = {x: x_batch, 
                          y_true: y_true_batch}

            # Run graph.
            _, c, _, summary = sess.run([optimizer, cost, accuracy, merged_summary_op],
                                          feed_dict=feed_train)
            
            # Compute average loss
            avg_cost += c / total_batch
            
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            acur = sess.run(accuracy, feed_dict=feed_train)
            print("Epoch: ", '%02d' % (epoch+1), " ===> Loss=", "{:.5f}".format(avg_cost), ", Training acurracy=", "{:.2%}".format(acur))
   
    # Ending time.
    end_time = time.time()
    
    # Difference between start and end-times.
    dif = end_time - start_time

    # Print the time-usage.
    print('\nTime usage: ' + str(timedelta(seconds=int(round(dif)))))
    
    pass

In [17]:
# Split the test-set into smaller batches of this size.
test_batch_size = 256

def test_accuracy():

    # Number of images in the test-set.
    num_test = len(mnist.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = mnist.test.images[i:j, :]

        # Get the associated labels.
        labels = mnist.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = np.argmax(mnist.test.labels, axis=1)

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    print('\n\nAccuracy Test Set: {0:.1%} ({1} / {2})'.format(acc, correct_sum, num_test))

In [18]:
with tf.Session() as sess:
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter('lab_2/log_files', graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 1.73335 , Training acurracy= 71.88%
Epoch:  02  ===> Loss= 0.86090 , Training acurracy= 72.66%
Epoch:  03  ===> Loss= 0.72863 , Training acurracy= 70.31%
Epoch:  04  ===> Loss= 0.56721 , Training acurracy= 87.50%
Epoch:  05  ===> Loss= 0.40811 , Training acurracy= 84.38%
Epoch:  06  ===> Loss= 0.37747 , Training acurracy= 82.81%
Epoch:  07  ===> Loss= 0.35909 , Training acurracy= 89.84%
Epoch:  08  ===> Loss= 0.14375 , Training acurracy= 97.66%
Epoch:  09  ===> Loss= 0.11789 , Training acurracy= 99.22%
Epoch:  10  ===> Loss= 0.10678 , Training acurracy= 95.31%
Epoch:  11  ===> Loss= 0.09950 , Training acurracy= 97.66%
Epoch:  12  ===> Loss= 0.09228 , Training acurracy= 99.22%
Epoch:  13  ===> Loss= 0.08592 , Training acurracy= 97.66%
Epoch:  14  ===> Loss= 0.08196 , Training acurracy= 99.22%
Epoch:  15  ===> Loss= 0.07795 , Training acurracy= 100.00%
Epoch:  16  ===> Loss= 0.07332 , Training acurracy= 100.00%
Epoch:  17  ===> Loss= 0.07046 , Training acurracy= 10

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

The respective images can be found in the folder. They are not displayed becuase we noticed an issue when we exported as html.

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |       0.9850       |      0.7745         |       
| Training Time        |       0:16:06      |      0:15:01        | 

- Which optimizer gives the best accuracy on test data?<br>
According to the results, both methods give us similar results. In terms of accuracy, anyone gives similar results. In our opinion you cannot compare accuracy first because they start randomly at differents points and second because AdamOptimizer includes momentum in its algorithm to speed up calculation. The strage thing here is why both method reach similar end times. Probably becuase the load in the ZOE system does not allow it to compare correctly or maybe becayse AdamOptimizer needs to compute more things internatlly before to get the right momentum. 


In [19]:
tf.reset_default_graph()

In [20]:
'''Input variables'''
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

'''CNN Tensorflow architecture'''
# Convolutional layer 2
layer_conv1, weights_conv1 = LeNet5_Model(input=x_image,
                                           num_input_channels=num_channels,
                                           filter_size=filter_size_1,
                                           num_filters=num_filters_1,
                                           padding=padding_1,
                                           activ_pooling=True)
# Convolutional layer 2
layer_conv2, weights_conv2 = LeNet5_Model(input=layer_conv1,
                                           num_input_channels=num_filters_1,
                                           filter_size=filter_size_2,
                                           num_filters=num_filters_2,
                                           padding=padding_2,
                                           activ_pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Fully Connectes layer 1
fc_layer_1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size_1)


# Fully Connectes layer 2
fc_layer_2 = new_fc_layer(input=fc_layer_1,
                         num_inputs=fc_size_1,
                         num_outputs=fc_size_2)


# Fully Connectes layer 3
fc_layer_3 = new_fc_layer(input=fc_layer_2,
                         num_inputs=fc_size_2,
                         num_outputs=num_classes)

In [21]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

In [22]:
accuracy = evaluate(y_pred_cls, y_true_cls)

In [23]:
# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3,
                                                        labels=y_true)
# cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Loss_Adam'):
    cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Accuracy_Adam'):
    accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [24]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar('Cost_Adam', cost)

# Create a summary to monitor accuracy tensor
tf.summary.scalar('Accuracy_Adam', accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [26]:
with tf.Session() as sess:    
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 1.63065 , Training acurracy= 36.72%
Epoch:  02  ===> Loss= 1.41042 , Training acurracy= 39.84%
Epoch:  03  ===> Loss= 1.40150 , Training acurracy= 41.41%
Epoch:  04  ===> Loss= 1.29377 , Training acurracy= 52.34%
Epoch:  05  ===> Loss= 1.17829 , Training acurracy= 56.25%
Epoch:  06  ===> Loss= 1.16743 , Training acurracy= 50.78%
Epoch:  07  ===> Loss= 1.17000 , Training acurracy= 56.25%
Epoch:  08  ===> Loss= 1.16721 , Training acurracy= 46.88%
Epoch:  09  ===> Loss= 1.16876 , Training acurracy= 43.75%
Epoch:  10  ===> Loss= 1.16363 , Training acurracy= 55.47%
Epoch:  11  ===> Loss= 1.16558 , Training acurracy= 58.59%
Epoch:  12  ===> Loss= 1.16526 , Training acurracy= 53.12%
Epoch:  13  ===> Loss= 1.14774 , Training acurracy= 62.50%
Epoch:  14  ===> Loss= 0.95410 , Training acurracy= 69.53%
Epoch:  15  ===> Loss= 0.68768 , Training acurracy= 71.09%
Epoch:  16  ===> Loss= 0.50476 , Training acurracy= 76.56%
Epoch:  17  ===> Loss= 0.49570 , Training acurracy= 76.5

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [27]:
def LeNet5_Model_Dropout(input,                # Input layer.
                         num_input_channels,   # Channels in previous layer.
                         filter_size,          # Width and height filter.
                         num_filters,          # Number of filters.
                         padding,              # Padding approach.
                         activ_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Initialize weight given shape.
    weights = weight_variable(shape=shape)

    # Initialize bias given shape, one for each filter.
    biases = bias_variable(shape=num_filters)

    # Convolution operation.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding=padding)

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling according to LeNet-5 architecture.
    if activ_pooling:
        # This is 2x2 max-pooling, which applies
        # in all cases of CNN architecture.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='VALID')

    # Layer dropout with keep_prob = 0.75
    layer = tf.nn.dropout(layer, keep_prob = 0.75)
    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)

    return layer, weights

tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

'''Input variables'''
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

'''CNN Tensorflow architecture'''
# Convolutional layer 2
layer_conv1, weights_conv1 = LeNet5_Model(input=x_image,
                                           num_input_channels=num_channels,
                                           filter_size=filter_size_1,
                                           num_filters=num_filters_1,
                                           padding=padding_1,
                                           activ_pooling=True)
# Convolutional layer 2
layer_conv2, weights_conv2 = LeNet5_Model_Dropout(input=layer_conv1,
                                           num_input_channels=num_filters_1,
                                           filter_size=filter_size_2,
                                           num_filters=num_filters_2,
                                           padding=padding_2,
                                           activ_pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Fully Connectes layer 1
fc_layer_1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size_1)


# Fully Connectes layer 2
fc_layer_2 = new_fc_layer(input=fc_layer_1,
                         num_inputs=fc_size_1,
                         num_outputs=fc_size_2)


# Fully Connectes layer 3
fc_layer_3 = new_fc_layer(input=fc_layer_2,
                         num_inputs=fc_size_2,
                         num_outputs=num_classes)

In [28]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

In [29]:
accuracy = evaluate(y_pred_cls, y_true_cls)

In [30]:
# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3,
                                                        labels=y_true)
# cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Loss_Drop'):
    cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Accuracy_Drop'):
    accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [31]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar('Cost_Drop', cost)

# Create a summary to monitor accuracy tensor
tf.summary.scalar('Accuracy_Drop', accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initializing the variables
no_iter = 0

In [32]:
with tf.Session() as sess:
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 1.93373 , Training acurracy= 57.81%
Epoch:  02  ===> Loss= 0.86281 , Training acurracy= 74.22%
Epoch:  03  ===> Loss= 0.63920 , Training acurracy= 85.16%
Epoch:  04  ===> Loss= 0.54896 , Training acurracy= 87.50%
Epoch:  05  ===> Loss= 0.49489 , Training acurracy= 86.72%
Epoch:  06  ===> Loss= 0.46134 , Training acurracy= 92.19%
Epoch:  07  ===> Loss= 0.43528 , Training acurracy= 85.94%
Epoch:  08  ===> Loss= 0.41449 , Training acurracy= 84.38%
Epoch:  09  ===> Loss= 0.39937 , Training acurracy= 90.62%
Epoch:  10  ===> Loss= 0.38671 , Training acurracy= 86.72%
Epoch:  11  ===> Loss= 0.37641 , Training acurracy= 89.84%
Epoch:  12  ===> Loss= 0.36817 , Training acurracy= 91.41%
Epoch:  13  ===> Loss= 0.35923 , Training acurracy= 92.19%
Epoch:  14  ===> Loss= 0.35567 , Training acurracy= 87.50%
Epoch:  15  ===> Loss= 0.34811 , Training acurracy= 89.84%
Epoch:  16  ===> Loss= 0.34364 , Training acurracy= 90.62%
Epoch:  17  ===> Loss= 0.34052 , Training acurracy= 87.5